
=================================================

Milestone 3

Nama  : Sinthiya Kusuma Nagari

Batch : FTDS-010-HCK

Objective: Program ini dibuat untuk melakukan automatisasi transform dan load data dari PostgreSQL ke ElasticSearch. Adapun dataset yang dipakai adalah dataset mengenai penjualan dari bisnis retail di US tahun 2011-2014

=================================================

In [1]:
# import libraries
import great_expectations as gx
import pandas as pd
from great_expectations.checkpoint import Checkpoint

In [2]:
# mendefine environmet mengambil context di environtment gx
context = gx.get_context()

In [3]:
# mendefine datasource
datasource = context.sources.add_pandas(name="datasource1")

In [4]:
# membaca data csv
df=pd.read_csv('P2M3_Sinthiya_data_clean.csv')
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [5]:
#mendefine nama, data asset, dan my batch request
name = "superstore"
data_asset = datasource.add_dataframe_asset(name=name)
my_batch_request = data_asset.build_batch_request(dataframe=df)

In [6]:
# Membuat expectation suite
expectation_suite_name = 'expectation-superstore-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Membuat  validator menggunakan expectation suite di atas
validator = context.get_validator(
    batch_request = my_batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check tvalidator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:
# Expectation 1 : Kolom `row _id` harus unik

validator.expect_column_values_to_be_unique('row_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Pada dataset ini hanya kolom row_id yang harus unik tidak boleh ada duplikasi

In [8]:
# Expectation 2 : Column `discount` harus kurang dari 1

validator.expect_column_values_to_be_between(
    column='discount', min_value=0, max_value=1
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Kolom diskon harus memiliki nilai lebih dari 0 dan kurang dari 1, karena diskon hanya berlaku dalam kisaran tersebut.

In [9]:
# Expectation 3: Kolom region hanya bisa terisi dengan:  "South", "West", "Central", "East"
validator.expect_column_values_to_be_in_set("region", ["South", "West","Central","East"])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
# Expectation 4: Kolom sales harus memiliki tipe integer atau float, tidak boleh bertipe object
validator.expect_column_values_to_be_in_type_list('sales', ['integer', 'float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [11]:
# Expectation 5: Jumlah panjang karakter daru kolom customer_id harus berjumlah 8
validator.expect_column_value_lengths_to_equal(column='customer_id', value=8)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [12]:
# Expectation 6: kolom order date harus memiliki format yang sama yaitu '%Y-%m-%d'
validator.expect_column_values_to_match_strftime_format(column="order_date", strftime_format='%Y-%m-%d')


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
# Expectation 7: tanggal pada kolom ship date harus lebih besar atau sama dengan order date
validator.expect_column_pair_values_a_to_be_greater_than_b(column_A="ship_date", column_B="order_date", or_equal=True)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9994,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tanggal pada kolom ship date harus lebih besar atau sama dengan order date, karena akan ada tanggal pengiriman apabila sudah melakukan pemesanan.

In [14]:
# Simpan ke Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

In [15]:
# Checkpoint
checkpoint = context.add_or_update_checkpoint(
    name='checkpoint',
    validator=validator
)

In [16]:
# hasil cek point
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/45 [00:00<?, ?it/s]

Berdasarkan hasil cek point semua expectation bernilai true dan sukses 

In [17]:
# Membuat data docs
context.build_data_docs()

{'local_site': 'file://C:\\Users\\Shintya\\AppData\\Local\\Temp\\tmph3ixlxzf\\index.html'}

Apabila ingin melihat rincian keberhasilan untuk setiap expectation dapat menuju halaman berikut 

{'local_site': 'file://C:\\Users\\Shintya\\AppData\\Local\\Temp\\tmph3ixlxzf\\index.html'}